# Google Drive Connection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/263-NLP/Final_Project/

/content/drive/MyDrive/263-NLP/Final_Project


# Imports

In [ ]:
!pip install stop_words

In [ ]:
import pandas as pd
import numpy as np
import random
import collections
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from nltk import word_tokenize
from nltk.corpus import wordnet     
from nltk.stem import WordNetLemmatizer
from string import punctuation
stop_words_skt = text.ENGLISH_STOP_WORDS
import matplotlib.pyplot as plt

# Dataset

In [ ]:
train_path = 'data/data_w_title/train_cleaned_with head.csv'
train_data = pd.read_csv(train_path)

# test_path = 'data/evaluation.csv'
# test_data = pd.read_csv(test_path)

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
cnt1 = collections.Counter(train_data['url1_lang'])
print(cnt1)
cnt2 = collections.Counter(train_data['url2_lang'])
print(cnt2)

Counter({'en': 1661, 'de': 1362, 'es': 510, 'tr': 456, 'pl': 325, 'ar': 266, 'fr': 69})
Counter({'en': 2197, 'de': 826, 'es': 510, 'tr': 456, 'pl': 325, 'ar': 266, 'fr': 69})


In [ ]:
cnt_overall = collections.Counter(train_data['Overall'])
print(cnt_overall)

Counter({4.0: 1598, 1.0: 789, 3.0: 752, 2.0: 681, 3.6666666666666665: 187, 3.333333333333333: 123, 2.333333333333333: 100, 1.3333333333333333: 91, 2.6666666666666665: 84, 1.6666666666666667: 79, 3.5: 54, 1.5: 39, 2.5: 37, 3.75: 7, 1.75: 4, 2.75: 3, 3.8: 3, 3.4: 3, 2.25: 3, 3.25: 2, 1.25: 1, 3.875: 1, 1.4: 1, 2.2: 1, 3.571428571428572: 1, 2.625: 1, 3.6: 1, 3.8333333333333335: 1, 2.8: 1, 1.2: 1})


In [ ]:
print(len(train_data))
match = []
for i in range (len(train_data)):
    if train_data['url1_lang'][i] != train_data['url2_lang'][i]:
        match.append(i)
print(len(match))

4649
536


SEED

In [ ]:
np.random.seed(42)
random.seed(42)

# Data Cleaning

In [ ]:
def convertLabel(label):
    if label - 1 > 0 and label - 1 < 0.5:
        label = 1
    elif label - 2 >= -0.5 and label - 2 < 0.5:
        label = 2
    elif label - 3 >= -0.5 and label - 3 < 0.5:
        label = 3
    elif label - 4 >= -0.5:
        label = 4
    return label

3


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
import re
import string
from stop_words import get_stop_words
stopwords_lang = get_stop_words('english') + get_stop_words('german') + get_stop_words('spanish') + get_stop_words('turkish') + get_stop_words('polish') + get_stop_words('arabic') + get_stop_words('french')
combined_stopwords = set.union(set(stopwords_lang),set(punctuation),set(stop_words_skt))
# def clean(text):
#     text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
#     texter = re.sub(r"<br />", " ", text)
#     texter = re.sub(r"&quot;", "\"",texter)
#     texter = re.sub('&#39;', "\"", texter)
#     texter = re.sub('\n', " ", texter)
#     texter = re.sub(' u '," you ", texter)
#     texter = re.sub('`',"", texter)
#     texter = re.sub('_',"", texter)
#     texter = re.sub(' +', ' ', texter)
#     texter = re.sub(r"(!)\1+", r"!", texter)
#     texter = re.sub(r"(\?)\1+", r"?", texter)
#     texter = re.sub('&amp;', 'and', texter)
#     texter = re.sub('\r', ' ',texter)
#     texter = texter.lower()
#     texter = re.sub(r'\d+', '', texter)
#     clean = re.compile('<.*?>')
#     texter = texter.encode('ascii', 'ignore').decode('ascii')
#     texter = [word for word in texter if not word in combined_stopwords]
#     texter = ''.join(filter(str.isalpha, texter))
#     texter = re.sub(clean, '', texter)
#     if texter == "":
#       texter = ""
#     return texter
def clean(text):
    text = re.sub(r"\bhttp"," ",text)
    text = re.sub(r"[^a-zA-Z]", " ", text)
    text = re.sub("\d"," ",text)
    text = re.sub("\W"," ",text)
    text = text.lower()
    text = text.translate(str.maketrans("","", string.punctuation))
    text = text.split()
    text = [word for word in text if not word in combined_stopwords]
    text = ' '.join(filter(str.isalpha, text))
    return text

# test['text']= test_data['text'].apply(clean)

In [ ]:
# train_data['text1']= train_data['text1'].apply(clean)
err_occur = []
for i in (train_data.index):
    try: 
        train_data['text1'][i] = clean(train_data['text1'][i])
        train_data['text2'][i] = clean(train_data['text2'][i])
    except:
        err_occur.append(i)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
err_occur_head = []
for i in (train_data.index):
    try: 
        train_data['head1'][i] = clean(train_data['head1'][i])
        train_data['head2'][i] = clean(train_data['head2'][i])
    except:
        err_occur_head.append(i)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
print(len(err_occur))

573


In [ ]:
train_data_clean = train_data.drop(train_data.index[err_occur])

In [ ]:
print(len(train_data_clean))

4076


# Train Test Split

In [ ]:
reduced_train_data = train_data_clean[["head1","text1","head2","text2","Overall"]]

In [ ]:
reduced_train_data['Overall']= reduced_train_data['Overall'].apply(convertLabel)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
reduced_train_data.to_csv("data/holy_data/clean_af_data_with_head.csv")

In [ ]:
train, test = train_test_split(reduced_train_data, test_size=0.2)

In [ ]:
print(train)

                                                                             head1  \
3808                                             corona schlagermove hamburg f llt   
2576                            drama familia beb desaparecido miami ruega regreso   
94                                            iran summons swiss envoy accusations   
1105                                 gallery baby love zoos celebrate new arrivals   
1824                                                 gran xito san silvestre aldea   
...                                                                            ...   
1172               plumes uncertainty hopes connectivity dot indian aviation space   
1340  new year begins chilly note north india mercury drops degrees celsius places   
894                                                                            NaN   
4011                           viktor orban kn pft fidesz verbleib evp bedingungen   
3651                                     corona kontro

In [ ]:
cnt_overall1 = collections.Counter(train['Overall'])
print(cnt_overall1)

Counter({4.0: 1304, 3.0: 710, 2.0: 655, 1.0: 591})
